## Built in tools

In [3]:
# DuckDuckGo search
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke("What is the current weather in new delhi?")

print(result)

c:\Users\Arbaz Khan\Desktop\LangChain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:62: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


New Delhi, India - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data. Current weather in New Delhi and forecast for today, tomorrow, and next 14 days Get today's real-time weather updates in New Delhi with hourly temperatures and a weekly forecast. Find out about New Delhi's temperature trends, rain chances, air quality (AQI), and humidity ... Latest weather forecast for Delhi for today's, hourly weather forecast, including today's temperatures in Delhi, wind, rain and more. Current weather in Delhi is 37°C. Get today's Delhi weather report along with accurate forecast including hourly, weekly and monthly reports at Oneindia.


In [6]:
# Shell tool
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

result = shell_tool.invoke("dir /s")

print(result)

Executing command:
 dir /s
 Volume in drive C has no label.
 Volume Serial Number is FC0B-D5FF

 Directory of c:\Users\Arbaz Khan\Desktop\LangChain\GenAI\Agents

01-07-2025  12:20    <DIR>          .
01-07-2025  12:20    <DIR>          ..
01-07-2025  12:57             3,723 1_tools.ipynb
               1 File(s)          3,723 bytes

     Total Files Listed:
               1 File(s)          3,723 bytes
               2 Dir(s)  10,225,811,456 bytes free



c:\Users\Arbaz Khan\Desktop\LangChain\venv\Lib\site-packages\langchain_community\tools\shell\tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom tools - tool decorator

In [7]:
from langchain_community.tools import tool

In [8]:
@tool
def multiply(num1: int, num2: int) -> int:
    """This function multiplies 2 numbers"""
    return num1 * num2

In [10]:
result = multiply.invoke({"num1": 9, "num2": 10})
result

90

In [29]:
print("Name of the tool:", multiply.name)
print("Description:", multiply.description)
print("Arguments:", multiply.args)
print("Json Schema:", multiply.args_schema.schema_json())

Name of the tool: multiply
Description: This function multiplies 2 numbers
Arguments: {'num1': {'title': 'Num1', 'type': 'integer'}, 'num2': {'title': 'Num2', 'type': 'integer'}}
Json Schema: {"title": "multiplySchema", "description": "This function multiplies 2 numbers", "type": "object", "properties": {"num1": {"title": "Num1", "type": "integer"}, "num2": {"title": "Num2", "type": "integer"}}, "required": ["num1", "num2"]}


### Structured tool

In [17]:
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field

In [18]:
class MultiplyInput(BaseModel):
    num1: int = Field("first number for multiply")
    num2: int = Field("second number for multiply")

In [19]:
def func_multiply(num1, num2):
    return num1 * num2

In [20]:
multiply_tool = StructuredTool.from_function(
    func=func_multiply,
    name="multiply",
    description="multiply 2 numbers",
    agrs_schema=MultiplyInput
)

In [22]:
result = multiply_tool.invoke({"num1": 9, "num2": 10})
result

90

In [30]:
print("Name of the tool:", multiply_tool.name)
print("Description:", multiply_tool.description)
print("Arguments:", multiply_tool.args)
print("Json Schema:", multiply_tool.args_schema.schema_json())

Name of the tool: multiply
Description: multiply 2 numbers
Arguments: {'num1': {'title': 'Num1'}, 'num2': {'title': 'Num2'}}
Json Schema: {"title": "multiplySchema", "type": "object", "properties": {"num1": {"title": "Num1"}, "num2": {"title": "Num2"}}, "required": ["num1", "num2"]}


### BaseTool

In [32]:
from langchain_community.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

In [33]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [34]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "multiply two numbers"

    args_schema: Type[BaseTool] = MultiplyInput
    def _run(self, a: int, b: int) -> int:
        return a * b

In [36]:
multiply_tool = MultiplyTool()
result = multiply_tool.invoke({'a':9, 'b':10})
print(result)

90


In [37]:
print("Name of the tool:", multiply_tool.name)
print("Description:", multiply_tool.description)
print("Arguments:", multiply_tool.args)
print("Json Schema:", multiply_tool.args_schema.schema_json())

Name of the tool: multiply
Description: multiply two numbers
Arguments: {'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}
Json Schema: {"properties": {"a": {"description": "The first number to add", "required": true, "title": "A", "type": "integer"}, "b": {"description": "The second number to add", "required": true, "title": "B", "type": "integer"}}, "required": ["a", "b"], "title": "MultiplyInput", "type": "object"}


## Toolkit

In [38]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [40]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [41]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
